In [31]:
import os 
import re
import json
from groq import Groq
import datetime

In [32]:
# os.chdir("..")

In [33]:
base_data_path = r"C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data"

if os.path.exists(base_data_path):
    print("It exists")
else:
    print("Does not exist")

It exists


In [34]:
folders = [
    os.path.join(base_data_path, f) # Gives me the path so I can run datetime on it and get the most recent folder 
    for f in os.listdir(base_data_path) # Grabs all the folders in it
    if os.path.isdir(os.path.join(base_data_path, f)) # Validates that the path actually exists
]


In [35]:
latest_folder = max(folders, key=os.path.getctime)
print("Latest folder:", latest_folder)


Latest folder: C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data\research_on_java


## Temporary List to store data 

In [36]:
messages = []

In [37]:
tasks_path = os.path.join(latest_folder, "tasks.json")

with open(tasks_path, "r") as f:
    tasks = json.load(f)

In [38]:
retriever_agent = Groq()

In [39]:
search_results = [
    {
        "title": "Java Programming Language Overview",
        "url": "https://example.com/java-overview",
        "snippet": "Java is a high level object oriented programming language developed by Sun Microsystems."
    },
    {
        "title": "Key Features of Java",
        "url": "https://example.com/java-features",
        "snippet": "Java provides platform independence, automatic memory management, and strong security."
    }
]


In [40]:
retriever_system_template = """
You are a research retrieval agent.
Your job: Extract accurate facts from the provided search results for the following task:
Task: {task_description}

Rules:
- Use ONLY the given search results
- Do NOT invent information
- Keep summaries concise and factual
- Include the source URL for every item
- If results are insufficient, return an empty list

IMPORTANT: Your response must contain ONLY the <answer> block below. No extra text, no explanation, no preamble.

<answer>
[
  {{
    "source": "...",
    "title": "...",
    "summary": "...",
    "key_points": ["...", "..."]
  }}
]
</answer>

Do not write anything before or after the <answer> tags.
"""

In [41]:
for task in tasks:

    task_description = task["description"]

    retriever_system_prompt = retriever_system_template.format(
        task_description=task_description
    )

    search_results_str = json.dumps(search_results, indent=2)

    messages = [
        {"role": "system", "content": retriever_system_prompt},
        {"role": "user", "content": f"Search results:\n{search_results_str}"}
    ]

    completion = retriever_agent.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
        {"role": "system", "content": retriever_system_prompt},
        {"role": "user", "content": f"Search results:\n{search_results_str}"},
        {"role": "assistant", "content": "<answer>"}  # Forces model to continue from here
    ],
    stop=["</answer>"],
    stream=False,
  )

    reply = "<answer>" + completion.choices[0].message.content + "</answer>"
    print(reply)


<answer> 
[
  {
    "source": "https://example.com/java-overview",
    "title": "Java Programming Language Overview",
    "summary": "Java is a high-level object-oriented programming language developed by Sun Microsystems.",
    "key_points": ["Object-oriented", "Developed by Sun Microsystems"]
  },
  {
    "source": "https://example.com/java-features",
    "title": "Key Features of Java",
    "summary": "Java provides platform independence, automatic memory management, and strong security.",
    "key_points": ["Platform independence", "Automatic memory management", "Strong security"]
  }
]
</answer>
<answer> 
[
  {
    "source": "https://example.com/java-overview",
    "title": "Java Programming Language Overview",
    "summary": "Java is a high-level object-oriented programming language developed by Sun Microsystems.",
    "key_points": ["Object-oriented", "Developed by Sun Microsystems"]
  },
  {
    "source": "https://example.com/java-features",
    "title": "Key Features of Java",